In [2]:
# Importing all libraries required

import pandas as pd
import numpy as np
import tensorflow as tf
from keras import regularizers

Using TensorFlow backend.


In [3]:
# Getting data labelled
# !!! Note that the data has been previously cleaned in Excel, therefore there are no gaps and unforeseen signs !!!

data = pd.read_csv('D:/123.csv', header=None)
columnsNames = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'Pred']
data.columns = columnsNames

In [5]:
# Normalize to the maximum value for each of the feature

age = data.iloc[:, 0]
age = np.array(age)
age = age / 100
data.iloc[:, 0] = age.astype(float)

cp = data.iloc[:, 2]
cp = np.array(cp)
cp = cp / 4
data.iloc[:, 2] = cp.astype(float)

trestbps = data.iloc[:, 3]
trestbps = np.array(trestbps)
trestbps = trestbps / 200
data.iloc[:, 3] = trestbps.astype(float)

chol = data.iloc[:, 4]
chol = np.array(chol)
chol = chol / 564
data.iloc[:, 4] = chol.astype(float)

restecg = data.iloc[:, 6]
restecg = np.array(restecg)
restecg = restecg / 2
data.iloc[:, 6] = restecg.astype(float)

thalach = data.iloc[:, 7]
thalach = np.array(thalach)
thalach = thalach / 202
data.iloc[:, 7] = thalach.astype(float)

oldpeak = data.iloc[:, 9]
oldpeak = np.array(oldpeak)
oldpeak = oldpeak / 6.2
data.iloc[:, 9] = oldpeak.astype(float)

slope = data.iloc[:, 10]
slope = np.array(slope)
slope = slope / 3
data.iloc[:, 10] = slope.astype(float)

ca = data.iloc[:, 11]
ca = np.array(ca)
ca = ca / 3
data.iloc[:, 11] = ca.astype(float)

thal = data.iloc[:, 12]
thal = np.array(thal)
thal = thal / 7
data.iloc[:, 12] = thal.astype(float)

y_pred = data.iloc[:, -1]
y_pred = np.array(y_pred)
y_pred = y_pred / 4
data.iloc[:, -1] = y_pred.astype(float)


In [61]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,Pred
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,0.545421,0.676768,0.789562,0.658468,0.438564,0.144781,0.498316,0.740591,0.326599,0.170251,0.534231,0.225589,0.675806,0.236532
std,0.090497,0.468500,0.241215,0.088814,0.092194,0.352474,0.497457,0.113572,0.469761,0.188084,0.206062,0.312988,0.276947,0.308638
min,0.290000,0.000000,0.250000,0.470000,0.223404,0.000000,0.000000,0.351485,0.000000,0.000000,0.333333,0.000000,0.428571,0.000000
25%,0.480000,0.000000,0.750000,0.600000,0.374113,0.000000,0.000000,0.658416,0.000000,0.000000,0.333333,0.000000,0.428571,0.000000
50%,0.560000,1.000000,0.750000,0.650000,0.430851,0.000000,0.500000,0.757426,0.000000,0.129032,0.666667,0.000000,0.428571,0.000000
75%,0.610000,1.000000,1.000000,0.700000,0.489362,0.000000,1.000000,0.821782,1.000000,0.258065,0.666667,0.333333,1.000000,0.500000
max,0.770000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
# Shuffling the data in order to avoid noт-random patterns in initial data

matrix = data.iloc[:, :]
matrix = np.array(matrix)
np.random.shuffle(matrix)
print(matrix)

[[0.47       1.         0.75       ... 0.         0.42857143 0.25      ]
 [0.59       1.         0.25       ... 0.         0.42857143 0.25      ]
 [0.46       0.         1.         ... 0.         0.42857143 0.        ]
 ...
 [0.47       1.         0.75       ... 0.         0.42857143 0.        ]
 [0.54       1.         1.         ... 0.33333333 1.         0.75      ]
 [0.51       1.         1.         ... 0.         1.         0.25      ]]


In [24]:
# Dividing data by features and labels

x_data = matrix[:, :13]
y_data = matrix[:, -1]

In [25]:
# Encoding labels in vectors

def one_hot_encoding(y_data):
    list = []
    for i in range((y_data).shape[0]):
        if y_data[i] == 0:
            list.append([1, 0, 0, 0, 0,])
        if y_data[i] == 0.25:
            list.append([0, 1, 0, 0, 0,])
        if y_data[i] == 0.5:
            list.append([0, 0, 1, 0, 0,])
        if y_data[i] == 0.75:
            list.append([0, 0, 0, 1, 0,])
        if y_data[i] == 1:
            list.append([0, 0, 0, 0, 1,])
    return(np.array(list))

y_encoded = one_hot_encoding(y_data)
print(y_encoded.shape)
print(x_data.shape)

(297, 5)
(297, 13)


In [60]:
# Building model within "One-vs-All" algorithm.
# For better understanding check this out:
# https://www.coursera.org/learn/machine-learning/lecture/gFpiW/multiclass-classification

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(11, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(0.02)),
    tf.keras.layers.Dense(11, activation=tf.nn.relu, kernel_regularizer=regularizers.l2(0.02)),
    tf.keras.layers.Dense(5, activation=tf.nn.sigmoid)
])
model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_data, y_encoded, validation_split=0.3, epochs=200)

Train on 207 samples, validate on 90 samples
Epoch 1/200
207/207 [==============================] - 2s 8ms/step - loss: 1.1454 - acc: 0.4657 - val_loss: 1.1176 - val_acc: 0.5489
Epoch 2/200
207/207 [==============================] - 0s 178us/step - loss: 1.0967 - acc: 0.6039 - val_loss: 1.0770 - val_acc: 0.6911
Epoch 3/200
207/207 [==============================] - 0s 183us/step - loss: 1.0605 - acc: 0.7063 - val_loss: 1.0424 - val_acc: 0.7422
Epoch 4/200
207/207 [==============================] - 0s 193us/step - loss: 1.0283 - acc: 0.7643 - val_loss: 1.0096 - val_acc: 0.7867
Epoch 5/200
207/207 [==============================] - 0s 178us/step - loss: 0.9976 - acc: 0.7913 - val_loss: 0.9789 - val_acc: 0.7844
Epoch 6/200
207/207 [==============================] - 0s 188us/step - loss: 0.9686 - acc: 0.8029 - val_loss: 0.9493 - val_acc: 0.8000
Epoch 7/200
207/207 [==============================] - 0s 202us/step - loss: 0.9404 - acc: 0.8155 - val_loss: 0.9204 - val_acc: 0.8044
Epoch 8/200


Epoch 61/200
207/207 [==============================] - 0s 178us/step - loss: 0.4451 - acc: 0.8667 - val_loss: 0.4318 - val_acc: 0.8778
Epoch 62/200
207/207 [==============================] - 0s 202us/step - loss: 0.4436 - acc: 0.8696 - val_loss: 0.4297 - val_acc: 0.8667
Epoch 63/200
207/207 [==============================] - 0s 188us/step - loss: 0.4423 - acc: 0.8676 - val_loss: 0.4281 - val_acc: 0.8644
Epoch 64/200
207/207 [==============================] - 0s 178us/step - loss: 0.4410 - acc: 0.8696 - val_loss: 0.4272 - val_acc: 0.8711
Epoch 65/200
207/207 [==============================] - 0s 173us/step - loss: 0.4396 - acc: 0.8705 - val_loss: 0.4259 - val_acc: 0.8667
Epoch 66/200
207/207 [==============================] - 0s 178us/step - loss: 0.4385 - acc: 0.8657 - val_loss: 0.4252 - val_acc: 0.8711
Epoch 67/200
207/207 [==============================] - 0s 169us/step - loss: 0.4377 - acc: 0.8696 - val_loss: 0.4236 - val_acc: 0.8622
Epoch 68/200
207/207 [==========================

207/207 [==============================] - 0s 173us/step - loss: 0.3977 - acc: 0.8696 - val_loss: 0.3933 - val_acc: 0.8756
Epoch 122/200
207/207 [==============================] - 0s 183us/step - loss: 0.3967 - acc: 0.8696 - val_loss: 0.3938 - val_acc: 0.8733
Epoch 123/200
207/207 [==============================] - 0s 188us/step - loss: 0.3962 - acc: 0.8696 - val_loss: 0.3942 - val_acc: 0.8778
Epoch 124/200
207/207 [==============================] - 0s 173us/step - loss: 0.3959 - acc: 0.8676 - val_loss: 0.3918 - val_acc: 0.8756
Epoch 125/200
207/207 [==============================] - 0s 188us/step - loss: 0.3950 - acc: 0.8686 - val_loss: 0.3916 - val_acc: 0.8756
Epoch 126/200
207/207 [==============================] - 0s 207us/step - loss: 0.3950 - acc: 0.8686 - val_loss: 0.3908 - val_acc: 0.8756
Epoch 127/200
207/207 [==============================] - 0s 183us/step - loss: 0.3943 - acc: 0.8676 - val_loss: 0.3905 - val_acc: 0.8756
Epoch 128/200
207/207 [==============================] 

Epoch 181/200
207/207 [==============================] - 0s 193us/step - loss: 0.3751 - acc: 0.8725 - val_loss: 0.3760 - val_acc: 0.8800
Epoch 182/200
207/207 [==============================] - 0s 173us/step - loss: 0.3749 - acc: 0.8725 - val_loss: 0.3766 - val_acc: 0.8822
Epoch 183/200
207/207 [==============================] - 0s 193us/step - loss: 0.3747 - acc: 0.8696 - val_loss: 0.3766 - val_acc: 0.8822
Epoch 184/200
207/207 [==============================] - 0s 193us/step - loss: 0.3740 - acc: 0.8734 - val_loss: 0.3764 - val_acc: 0.8822
Epoch 185/200
207/207 [==============================] - 0s 169us/step - loss: 0.3741 - acc: 0.8705 - val_loss: 0.3747 - val_acc: 0.8711
Epoch 186/200
207/207 [==============================] - 0s 164us/step - loss: 0.3738 - acc: 0.8715 - val_loss: 0.3752 - val_acc: 0.8822
Epoch 187/200
207/207 [==============================] - 0s 164us/step - loss: 0.3740 - acc: 0.8705 - val_loss: 0.3749 - val_acc: 0.8822
Epoch 188/200
207/207 [==================

Total training accuracy: 0.8725
Total validation accuracy: 0.8822